# Growing Sphere

In [ ]:
#| default_exp methods.sphere

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *
from nbdev import show_doc
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#| export
from __future__ import annotations
from relax.import_essentials import *
from relax.methods.base import BaseCFModule
from relax.utils import *

In [ ]:
#| export
@partial(jit, static_argnums=(2, 5))
def hyper_sphere_coordindates(
    rng_key: jrand.PRNGKey, # Random number generator key
    x: Array, # Input instance with only continuous features. Shape: (1, n_features)
    n_samples: int, # Number of samples
    high: float, # Upper bound
    low: float, # Lower bound
    p_norm: int = 2 # Norm
):
    # Adapted from 
    # https://github.com/carla-recourse/CARLA/blob/24db00aa8616eb2faedea0d6edf6e307cee9d192/carla/recourse_methods/catalog/growing_spheres/library/gs_counterfactuals.py#L8
    key_1, key_2 = jrand.split(rng_key)
    delta = jrand.normal(key_1, shape=(n_samples, x.shape[-1]))
    dist = jrand.uniform(key_2, shape=(n_samples,)) * (high - low) + low
    norm_p = jnp.linalg.norm(delta, ord=p_norm, axis=1)
    d_norm = jnp.divide(dist, norm_p).reshape(-1, 1)  # rescale/normalize factor
    delta = jnp.multiply(delta, d_norm)
    candidates = x + delta

    return candidates

In [ ]:
#| export
@partial(jit, static_argnums=(1, 2))
def sample_categorical(rng_key: jrand.PRNGKey, col_size: int, n_samples: int):
    rng_key, subkey = jrand.split(rng_key)
    prob = jnp.ones(col_size) / col_size
    cat_sample = jax.nn.one_hot(
        jrand.categorical(rng_key, prob, shape=(n_samples,)), num_classes=col_size
    )
    return subkey, cat_sample


def cat_sample(
    rng_key: jrand.PRNGKey, # Random number generator key
    cat_array_sizes: List[int],  # A list of the number of categories for each categorical feature
    n_samples: int,  # Number of samples to sample
):  
    candidates = []
    for col in cat_array_sizes:
        rng_key, cat_sample = sample_categorical(rng_key, col, n_samples)
        candidates.append(cat_sample)
    candidates = jnp.concatenate(candidates, axis=1)
    
    return candidates

In [ ]:
#| export
@auto_reshaping('x')
def _growing_spheres(
    rng_key: jrand.PRNGKey, # Random number generator key
    x: Array, # Input instance. Shape: (n_features)
    pred_fn: Callable, # Prediction function
    n_steps: int, # Number of steps
    n_samples: int,  # Number of samples to sample
    cat_idx: int, # Index of categorical features
    cat_arrays: List[List[str]],  # A list of a list of each categorical feature name
    step_size: float, # Step size
    p_norm: int, # Norm
    apply_fn: Callable # Apply immutable constraints
):  
    @jit
    def dist_fn(x, cf):
        if p_norm == 1:
            return jnp.abs(cf - x).sum(axis=1)
        elif p_norm == 2:
            return jnp.linalg.norm(cf - x, ord=2, axis=1)
        else:
            raise ValueError("Only p_norm = 1 or 2 is supported")
    
    @loop_tqdm(n_steps)
    def step(i, state):
        candidate_cf, count, rng_key = state
        rng_key, subkey_1, subkey_2 = jrand.split(rng_key, num=3)
        low, high = step_size * count, step_size * (count + 1)
        # Sample around x
        cont_candidates = hyper_sphere_coordindates(subkey_1, x[:, :cat_idx], n_samples, high, low, p_norm)
        cat_candidates = cat_sample(subkey_2, cat_array_sizes, n_samples)
        candidates = jnp.concatenate([cont_candidates, cat_candidates], axis=1)
        # Apply immutable constraints
        candidates = apply_fn(x=x, cf=candidates)
        assert candidates.shape[1] == x.shape[1], f"candidates.shape = {candidates.shape}, x.shape = {x.shape}"

        # Calculate distance
        dist = dist_fn(x, candidates)

        # Calculate counterfactual labels
        candidate_preds = pred_fn(candidates).round().reshape(-1)
        indices = jnp.where(candidate_preds != y_pred, 1, 0).astype(bool)

        candidates = jnp.where(indices.reshape(-1, 1), 
                               candidates, jnp.ones_like(candidates) * jnp.inf)
        dist = jnp.where(indices.reshape(-1, 1), dist, jnp.ones_like(dist) * jnp.inf)

        closest_idx = dist.argmin()
        candidate_cf_update = candidates[closest_idx].reshape(1, -1)

        candidate_cf = jnp.where(
            dist[closest_idx].mean() < dist_fn(x, candidate_cf).mean(),
            candidate_cf_update, 
            candidate_cf
        )
        return candidate_cf, count + 1, rng_key
    
    y_pred = pred_fn(x).round().reshape(-1)
    candidate_cf = jnp.ones_like(x) * jnp.inf
    cat_array_sizes = [len(cat_array) for cat_array in cat_arrays]
    count = 0
    state = (candidate_cf, count, rng_key)
    candidate_cf, _, _ = lax.fori_loop(0, n_steps, step, state)
    # if `inf` is found, return the original input
    candidate_cf = jnp.where(jnp.isinf(candidate_cf), x, candidate_cf)
    return candidate_cf

In [ ]:
#| export
def apply_immutable(x: Array, cf: Array, immutable_idx: List[int]):
    if immutable_idx is not None:
        cf = cf.at[:, immutable_idx].set(x[:, immutable_idx])
    return cf

In [ ]:
#| export
class GSConfig(BaseParser):
    seed: int = 42
    n_steps: int = 100
    n_samples: int = 1000
    step_size: float = 0.05
    p_norm: int = 2
    

In [ ]:
#| export
class GrowingSphere(BaseCFModule):
    name = "Growing Sphere"

    def __init__(
        self,
        configs: Dict | GSConfig = None
    ):
        if configs is None:
            configs = GSConfig()
        self.configs = validate_configs(configs, GSConfig)
        self.rng = jrand.PRNGKey(self.configs.seed)
    
    def generate_cf(
        self,
        x: Array,
        pred_fn: Callable,
    ):
        cat_idx = self.data_module.cat_idx
        apply_immutable_partial = partial(
            apply_immutable, immutable_idx=self.data_module._imutable_idx_list)
        cf = _growing_spheres(
            self.rng,
            x,
            pred_fn,
            self.configs.n_steps,
            self.configs.n_samples,
            cat_idx,
            self.data_module._cat_arrays,
            self.configs.step_size,
            self.configs.p_norm,
            apply_immutable_partial,
        )
        return cf
    
    def generate_cfs(
        self, 
        X: Array, 
        pred_fn: Callable = None
    ) -> jnp.ndarray:
        rng_keys = jrand.split(jrand.PRNGKey(self.configs.seed), num=X.shape[0])
        generate_cf_partial = jit(partial(self.generate_cf, pred_fn=pred_fn))
        cfs = jax.vmap(generate_cf_partial)(X, rng_keys)
        # cfs = generate_cf_partial(X[0], rng_keys[0])
        return cfs

#### Test

In [ ]:
from relax.data import load_data
from relax.module import PredictiveTrainingModule, PredictiveTrainingModuleConfigs
from relax.evaluate import generate_cf_explanations, benchmark_cfs
from relax.trainer import train_model

In [ ]:
dm = load_data('adult', data_configs=dict(sample_frac=0.1))
# dm = load_data('adult',)

In [ ]:
m_config = dict(sizes=[50, 10, 50], lr=0.03)
t_config = dict(n_epochs=5, batch_size=256)

training_module = PredictiveTrainingModule(m_config)
params, opt_state = train_model(
    training_module, dm, t_config
)
# predict function
# pred_fn = lambda x: training_module.forward(params, x, is_training=False)
pred_fn = lambda x, params, key: training_module.forward(
    params, key, x, is_training=False
)


/home/birk/code/ReLax/relax/_ckpt_manager.py:47: UserWarning: `monitor_metrics` is not specified in `CheckpointManager`. No checkpoints will be stored.
  warnings.warn(
Epoch 4: 100%|██████████| 10/10 [00:00<00:00, 255.62batch/s, train/train_loss_1=0.072]


In [ ]:
#| slow
gs = GrowingSphere({'n_steps': 50, 'n_samples': 100, 'step_size': 0.1})
# gs.hook_data_module(dm)

In [ ]:
#| slow
cf_exp = generate_cf_explanations(
    gs, dm, pred_fn=pred_fn, 
    pred_fn_args=dict(
        params=params, key=random.PRNGKey(0)
    )
)
assert not np.array_equal(cf_exp.cfs[0], cf_exp.cfs[1])

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
#| slow
benchmark_cfs([cf_exp])

,,acc,validity,proximity
adult,Growing Sphere,0.823855,1.0,7.295473


In [ ]:
#| hide
gs = GrowingSphere({'n_steps': 5, 'n_samples': 10, 'step_size': 0.1})
cf_exp = generate_cf_explanations(
    gs, dm, pred_fn=pred_fn, 
    pred_fn_args=dict(
        params=params, key=random.PRNGKey(0)
    )
)
assert not np.array_equal(cf_exp.cfs[0], cf_exp.cfs[1])

  0%|          | 0/5 [00:00<?, ?it/s]